# 13장 - 세그멘테이션을 활용한 의심 결절 탐색

In [1]:
# 앞에서 실제로 작동하는 분류기 제작했지만 결절 후보 정보가 필요한 인위적인 환경에서 동작함
# 모델에 전체 CT인 중첩된 32*32*32 패치를 통째로 넣으면 에노테이션 달린 샘플의 10배인 CT당 31*31*7=6727 패치가 됨
# 경계면은 겹치고 분류기는 결절 후보가 중앙에 위치한다고 생각하므로 위치를 일정하게 잡지 않으면 문제가 발생

## 1절 - 프로젝트에 두 번째 모델 추가하기

In [2]:
# 분류기가 어디를 봐야 할지 알려주는 단계 수행 위해 원본 CT 스캔을 받아 결절처럼 보이는 모든 부분 찾기
# 결절일 가능성이 있는 지점 찾아 해당 복셀 표시하는 작업을 세그멘테이션(segmentation)이라 함
# 이 장 끝날 쯤에는 픽셀 단위로 레이블해주는 세그멘테이션 모델의 아키텍처 얻을 것임
# 수행하는 코드는 12장의 코드와 유사하며 작은 부분만 목표 삼아 바꿀 것임
# 새로운 입출력과 기타 요구사항에 맞추기 위해 모델, 데이터셋, 훈련 루프를 업데이트하고 새 모델 실행하며 결과를 평가 할 것임

In [3]:
# 13장 계획
# 1. 세그멘테이션: 유넷(U-Net) 모델 세그멘테이션이 어떻게 동작하는지 확인 후, 새 모델의 각 요소를 알아보고 세그멘테이션 과정을 진행하면서 어떤 일 발생하는지 확인
# 2. 업데이트: 세그멘테이션 구현은 기존 코드 베이스의 세 부분을 바꿔야 하는데 분류를 위해 작성했던 코드의 구조와 흡사하지만 세부사항이 다름
#   a. 모델 업데이트: 기존 유넷을 세그멘테이션 모델에 이식, 12장 모델은 참/거짓 분류 출력이지만 새 모델은 이미지 전체 출력
#   b. 데이터셋 변경: 데이터셋 수정해 CT 데이터 비트뿐 아니라 결절 마스크 정보도 제공함 / 세그멘테이션 훈련과 검증을 위해 CT 단면 전체와 2차원 크롭 필요
#   c. 훈련 루프 수정: 새 종류의 손실값 도입하고 최적화하도록 수정한 후, 텐서보드에서 세그멘테이션 결과를 확인하고 모델 가중치를 디스크에 저장

## 2절 - 다양한 세그멘테이션 유형

In [4]:
# 이 프로젝트에서는 시맨틱(semantic) 세그멘테이션 사용
# 레이블을 사용해 이미지의 개별 픽셀을 분류하는 방식으로 동작
# 수행되면 대상에 해당하는 영역을 구분하고 관심 영역을 레이블 마스크나 히트맵 형식으로 보여줌
# 만들 세그멘테이션은 단순 이진 레이블이므로, 결절이 의심되는 참값과 무관한 건강한 조직인 거짓값이 해당됨
# 이런 식으로 분류 신경망에 주입할 결절 후보 탐색

In [5]:
# 다른 접근 방식 확인
# 인스턴스(instance) 세그멘테이션 레이블 방식은 여러 구분된 레이블 사용하려 관심 객체를 개별적으로 레이블 함
# 결절이 서로 닿거나 겹쳐 있는 경우는 드믈고 개별 결절을 식별하기 위해서는 시맨틱이 나음
# 객체 탐지(object detection)는 이미지에서 관심있는 아이템 위치 주위에 경계 박스 그리는 방법
# 둘 다 유용한 방법이지만 구현이 복잡하고 배우는 입장에서 사용할 만한 최선의 방법이 아니며 객체 탐지 모델을 훈련에 많은 연산이 필요하기 때문에 적절하지 않음

## 3절 - 시맨틱 세그멘테이션: 픽셀 단위 분류

In [6]:
# 분류를 통해 이미지 속에 물체가 있는지 여부 판단 가능하고 세그멘테이션을 통해 물체가 이미지의 어느 부분에 있는지 알 수 있음

In [7]:
# 이미지를 출력해야 하므로 출력이 입력과 같은 크기이며 이를 위해서는 다른 모델 아키텍처가 필요함
# 간단한 세그멘테이션 모델로 다운샘플링 없는 연속적인 컨볼루션층 생각 가능
# 적절한 패딩만 주어지면 입력 크기와 동일한 출력을 결과로 만들지만 작은 컨볼루션층 여러 개가 겹치는 구조의 제한으로 수용 필드의 한계가 존재
# 분류 모델이 다음 컨볼루션에 두 배로 효과적인 전달을 위해 앞 단에 다운샘플링 계층을 두는 것처럼 필드 사이즈 늘리지 않으면 세그멘테이션된 픽셀은 매우 제한된 주위 영역에서만 이뤄짐
# 입력 픽셀과 일대일로 동일한 출력 픽셀 유지하면서 출력 픽셀의 수용 필드 개선을 위해 업샘플링(upsampling)으로 주어진 이미지의 해상도보다 높은 해상도의 이미지를 만들어야 함
# 선형 보간법이나 학습된 역컨볼루션(deconvolution)등의 옵션이 있지만 가장 단순하게 각 픽셀을 입력 픽셀과 동일한 값을 갖는 N*N 픽셀 블럭으로 바꾸기

#### .1 유넷 아키텍처

In [8]:
# 유넷이라 부르는 기본 세그멘테이션 알고리즘에 익숙해지는 1단계
# 유넷 아키텍처는 세그멘테이션을 위해 고안되었으며 픽셀 단위로 출력을 만들어 내는 신경망 설계

In [10]:
# 초기 신경망 설계부터 전체 크기의 컨볼루션 신경망이 가지는 수용 필드의 한계를 대응하기 위해 U자 모양 존재했엇음
# 이런 초기 신경망 설계에는 수렴 문제가 있었는데 다운샘플링 거치는 동안 공간 정보를 손실하기 때문에 매우 축소된 여러 영상에 정보가 다다르면 사물 경계의 정확한 위치를 인코딩해서 재구성하기 어려워졌었음
# 이를 해결하기 위해 유넷의 창안자들은 스킵 커넥션을 추가함
# 유넷에서 스킵 커넥션은 입력이 지나는 다운업샘플링 거치는 경로에 대한 지름길 역할
# 따라서 계층이 U자 모양 아래 부분의 넓은 수용 필드에서 나온 업샘플링 결과 받는 동시에 '복사 및 자르기' 브릿지 커넥션을 거치는 앞 단의 세부 계층의 출력도 받음
# 제일 마지막의 세부 계층이 두 경로에서 가장 나은 정보를 가지고 연산함으로써 인접한 영역에 대한 광범위한 관계 정보와 가장 높은 해상도 다루는 계층에서 온 세부 정보를 모두 가짐
# 신경망의 헤드에 위치한 제일 오른쪽의 1*1 컨볼루션층은 채널을 변환하는 것으로 분류 신경망의 완전 연결 계층과 유사하지만 픽셀 단위 변환인지 채널 단위 변환인지의 차이점을 가짐
# 이러한 방식으로 마지막 업샘플링 단계에서 사용된 필터의 개수가 출력에서 요구하는 분류 개수로 변환됨

## 4절 - 세그멘테이션을 위한 모델 업데이트

In [ ]:
# 코드 업데이트하고 모델 동작시키기
# 세그멘테이션 수행해 모든 픽셀에 대한 확률 출력하게 만들기

#### .1 기성품 모델을 프로젝트에 적용하기

In [11]:
# 기존 유넷 수정해 필요에 맞게 맞추기
# 순정 버전 모델에서 하나씩 없애 보면서 결과에 미치는 영향 보는 것은 좋은 경험(학계에서는 제거 연구(ablation study)라고 함)

In [12]:
# 첫 번째로 입력을 배치 정규화에 통과시켜 데이터셋을 직접 정규화할 필요 없음
# 동시에 개별 배치에 대한 정규화 통계값(평균, 표준편차)도 얻음
# 따라서 어떤 이유로 배치가 둔해지는, 즉 신경망에 들어가는 크롭된 CT에서 아무것도 확인할 수 없는 상황이 되면 비율을 더 크게 조정할 수 있음
# 모든 에포크에서 배치 샘플을 랜덤하게 선택하면 전체가 다 둔감한 배치가 되는 확률 줄이고 둔감한 샘플이 과장되는 것 방지함

In [13]:
# 두 번째로 출력값에 제한이 없으므로 출력을 nn.Sigmoid 계층에 통과시켜 출력 범위를 [0, 1]로 제한
# 세 번째로 모델의 전체 깊이와 필터 수를 사용할 만큼 줄일 것
# 표준 파라미터를 사용하는 모델의 용량은 우리가 사용하는 데이터셋 크기를 능가함 -> 사전 훈련된 모델 중에 필요에 딱 맞는 모델을 찾을 가능성은 없다는 의미
# 만드는 출력의 각 픽셀은 해당 픽셀이 결절의 일부인지에 대한 확률 나타내는 단일 채널이라는 것 기억해야 함

In [ ]:
# 다음의 유넷 래핑은 세 개의 속성으로 단순하게 구현된 것임
# 세 개의 속성은 각각에 대해 추가하려는 두 개의 피처와 미리 만들어진 모듈로 간주할 수 있는 유넷 자체임
# 받은 키워드 인자는 바로 유넷 생성자로 전달함
# model.py:17
class UNetWrapper(nn.Module):
    def __init__(self, **kwargs): # kwargs는 생성자에 전달한 키워드 인자 포함
        super().__init__()

        self.input_batchnorm = nn.BatchNorm2d(kwargs['in_channels']) # BatchNorm2d는 입력 채널 수를 입력받는데 키워드 인자에서 찾아 넣음
        self.unet = UNet(**kwargs) # 유넷이 일 진행하는 줄
        self.final = nn.Sigmoid()

        self._init_weights() # 분류기처럼 별도의 가중치 초기화 만들어 사용

In [ ]:
# nn.Sequential 인스턴스 사용할 수 있지만 코드와 스택 추적 시 가시성을 위해 명시적으로 forward 메소드 작성
# model.py:50
def forward(self, input_batch):
    bn_output = self.input_batchnorm(input_batch)
    un_output = self.unet(bn_output)
    fn_output = self.final(un_output)
    return fn_output

In [ ]:
# nn.BatchNorm2d 사용하는 점 주목
# 유넷은 기본저긍로 2차원 세그멘테이션 모델
# 구현을 적당히 변경해 3차원 컨볼루션에 적용하고 각 단면 정보 사용하도록 만들 수 있음
# 1차 구현은 메모리 사용이 꽤 높을 것이므로 CT 스캔을 더 잘게 쪼개야 할 수 있음
# Z 방향의 픽셀 간 거리가 평면 방향보다 커서 결절이 여러 단면에 걸쳐서 나타나는 경우가 덜함
# 이런 점 때문에 완전한 3차원 접근 방식은 목적에 덜 부합함
# 따라서 3차원 데이터를 단면으로 나누고 한 번에 한 단면에 대해 인접한 단면 제공
# 데이터를 2차원으로 표현하고 있는 상황이므로 채널을 활요하여 인접한 단면 나타내기
# 세 번째 차원에 대한 조치는 이미지에 완전 연결 계층 적용했을 때와 비슷하게 축 방향으로 버려지는 인접 관계를 모델이 다시 학습해야 하지만 찾는 구조의 크기가 작고 제한된 수의 단면이 제공되므로 모델이 재학습하기 어려운 과제는 아님

## 5절 - 세그멘테이션을 위한 데이터셋 업데이트

In [14]:
# CT 스캔과 애노테이션 데이터의 출처과 동일한 소스 데이터를 사용하지만 이번 모델은 다른 형태의 입력을 받고 다른 형태의 출력을 만들어 냄
# 원본 유넷 구현은 패딩된 컨볼루션 사용하지 않아 출력 세그멘테이션 맵이 입력보다 작았고 출력의 모든 픽셀은 완전하게 밀집된 수용 필드 가짐
# 출력 픽셀 하나하나에 대해 관계된 어떤 입력도 패딩되거나, 합성되거나, 불완전한 경우가 없어 어떤 크기의 이미지도 사용 가능했음

In [15]:
# 문제 해결을 위해 동일한 픽셀을 유지하는 접근 방식에는 컨볼루션과 다운샘플링 사이의 상호작용과 관련된 문제와 3차원 데이터의 특성과 관련된 문제가 있음

#### .1 매우 제한된 입력 크기를 갖는 유넷

In [16]:
# 유넷의 입출력 크기가 매우 제한적인 문제
# 다운샘플링 전후에 동일하게 컨볼루션 당 두 픽셀의 손실만 발생하게 하려면 입력을 특정 크기로 고정해야 함
# 유넷 논문에서는 572*572 이미지 패치 사용하고 388*388 출력 맵 크기를 가지는데 512*512 인 CT 단면보다 큰 입력과 작은 출력을 가져 경계에 가깝게 위치한 결절은 세그멘테이션 되지 않을 것임
# 보완하기 위해 유넷 생성자의 padding 플래그 파라미터에 True 값 줌
# 어떤 이미지든 사용 가능하고 동일한 크기의 출력 이미지 얻겠다는 의미
# 이미지 경계 부분에서 패딩된 영역이 수용 필드에 포함되어 정확도 떨어질 수 있지만 감수해야 함

#### .2 2차원vs 3차원 데이터의 유넷 사용시 장단점

In [17]:
# 다루는 3차원 데이터가 유넷의 2차원 입력에 맞지 않는다는 점
# 두 번째 다운샘플링 도달 전에 GPU 메모리 부족으로 제대로 동작하지 않음

In [18]:
# 각 단면을 2차원 세그멘테이션 문제로 보고 인접한 단면을 별로 채널로 제공해 3차원 정보에 대한 부분 보완할 것
# 이런 방식은 장단점 존재
# 채널로 표현하면 CT 단면 간 순서에 손실 있음
# 모든 채널이 컨볼루션 커널에 의해 선형적으로 병합되므로 두 단계 위나 아래 같은 관계 정보 잃게 됨
# 깊이 축에 해당하는 차원에서도 3차원 세그멘테이션을 다운샘플링하면서 얻는 넓은 수용 필드 효과도 잃게 됨
# 동일한 수의 데이터에 대해 CT 단명 방향은 행, 열 방향에 배히 더 넓은 범위 가지며, 일반적으로 결절은 제한된 수의 단면에 걸쳐 있으므로 이걸로 충분함

In [19]:
# 어떤 3차원 접근 방법 모두에서 현재 모델이 단면의 두께에 대해서 정확히 얼마인지 모르나든 점 고려해야 함
# 결국 모델이 다양한 두께의 단면 데이터를 학습함으로써 견고하게 대응하도록 만들어야 함

In [20]:
# 일반적으로 각 장단점에 대해 얼마만큼 타협해야 할 지 또는 어떤 부분을 타협해야 할지에 대한 정답은 없음
# 주의 깊은 실험이 핵심이며 거듭된 가설을 체계적으로 테스트함으로써 어떤 변화와 접근 방법이 문제에 효과적인지 파악하며 범위 좁혀 나가야 함

In [21]:
# 여러 번경을 가한 후 한번에 테스트하지 말 것은 반복 언급할 만큼 중요
# 변경 내용 중에는 다른 변경사항과 제대로 상호작용 못하는 것이 섞여 있을 가능성 큼
# 어떤 부분을 더 조사할지 뚜렷한 증거를 찾지 못하는 상황에 처하게 됨

#### .3 실측 데이터 만들기

In [22]:
# 첫 번째로 언급할 점은 사람이 레이블한 훈련 데이터와 모델에서 얻고자 하는 실제 출력이 서로 맞지 않음
# 애노테이션 데이터에 좌표 정보가 있지만 우리는 주어진 복셀이 결절 영역에 해당하는지를 알려주는 복셀 단위의 마스크 정보 필요
# 따라서 확보한 데이터로부터 마스크 직접 만들어야 하며 마스크 만드는 루틴이 정확한지 수동으로 검사해야 함

In [23]:
# 이처럼 수작업으로 휴리스틱 적용해 검증하는 방법은 규모를 키우기 어려운 점 있음
# 현재의 휴리스틱으로 모든 결절이 잘 처리됐는지 확인하기 위한 종합적인 방법은 고민하지 않을 것
# 지원이 충분하면 모든 경우를 수작업으로 검사하는 방식을 고민하겠지만 없으므로 단순히 몇 개의 샘플에 대해 출력 결과가 괜찮은지 확인하는 정도로만 진행

In [24]:
# 결과 확인을 위해 알고리즘이 진행되는 중간 단계를 쉽게 조사할 수 있도록 접근 방법과 API를 설계
# 이를 통해 많은 양의 튜플을 중간 결과값으로 출력하는 다소 투박한 함수들을 호출

In [25]:
# 바운딩 박스
# 결절의 위치를 변환해 결절 전체를 둘러싸는 바운딩 박스(bounding box)에 넣는 작업부터 시작(실제 결절에 대해서만 수행)
# 결절 모양의 대략적인 중심 부위가 결절의 위치라고 가정하면 결절의 중심부에서 각 차원별로 바깥쪽으로 저밀도 복셀을 만날 때까지 뻗어나가서 일반 폐 조직에 도달한 것을 나타낼 수 있음
# 결절의 중심점 위치라고 애노테이션이 달린 지점을 출발점으로 하여 복셀 탐색 시작
# 이후 열 축을 따라 중심점과 인접한 복셀의 밀도를 검사해 양 쪽 모두 고밀도 조직이면 거리를 늘려서 탐색 진행하고 한 쪽이라도 저밀도 복셀 발견하면 탐색 중지
# 행 축과 인덱스 축도 동일하게 탐색

In [26]:
# 복셀 바운딩 박스 구현 코드
# dsets.py:131
center_irc = xyz2irc(
        candidateInfo_tup.center_xyz,
        self.origin_xyz,
        self.vxSize_xyz,
        self.direction_a,
    )
    ci = int(center_irc.index)
    cr = int(center_irc.row)
    cc = int(center_irc.col)

    index_radius = 2
    try:
        while self.hu_a[ci + index_radius, cr, cc] > threshold_hu and \
                self.hu_a[ci - index_radius, cr, cc] > threshold_hu:
            index_radius += 1
    except IndexError: # 인덱스가 텐서 크기 넘어가는 경우 대비
        index_radius -= 1

    row_radius = 2
    try:
        while self.hu_a[ci, cr + row_radius, cc] > threshold_hu and \
                self.hu_a[ci, cr - row_radius, cc] > threshold_hu:
            row_radius += 1
    except IndexError:
        row_radius -= 1

    col_radius = 2
    try:
        while self.hu_a[ci, cr, cc + col_radius] > threshold_hu and \
                self.hu_a[ci, cr, cc - col_radius] > threshold_hu:
            col_radius += 1
    except IndexError:
        col_radius -= 1

In [27]:
# 양 쪽을 모두 경계 이하인지 확인하므로 결절 위치 정보가 상대적으로 실제 중심에 있어야 함
# 각 방향으로 탐색하면 다른 고밀도 조직과 인접한 결절에 경우 과도한 탐색을 하게 되므로 양쪽 방향을 동시에 탐색
# 탐색이 끝나면 바운딩 박스 마스킹 배열의 해당 박스 값을 True로 넣기

In [28]:
# 모든 기능을 함수 하나에 넣고 모든 결절에 대해 루프 돌 예정
# 각 결절에 대해 탐색을 수행하고 부리언 텐서 boundingBox_a에 바운딩 박스 찾았다고 표시
# 루프 끝나면 바운딩 박스 마스크가 경계값인 -700HU(0.3g/cc)보다 밀도 높은 조직에 겹치는 경우 박스의 모서리 잘라내어 결절의 윤곽 모양을 다듬을 것임

In [ ]:
# 결절 바운딩 박스 탐색 함수 코드
# dsets.py:127
def buildAnnotationMask(self, positiveInfo_list, threshold_hu = -700):
    boundingBox_a = np.zeros_like(self.hu_a, dtype=np.bool)

    for candidateInfo_tup in positiveInfo_list:
        # 복셀 바운딩 박스 구현 코드 위치

        # 169행
        boundingBox_a[
             ci - index_radius: ci + index_radius + 1,
             cr - row_radius: cr + row_radius + 1,
             cc - col_radius: cc + col_radius + 1] = True

    mask_a = boundingBox_a & (self.hu_a > threshold_hu)

    return mask_a

In [ ]:
# CT 초기화 때 마스크 생성 호출하기
# 결절 정보를 가진 튜플 리스트를 CT와 같은 형태의 '이것은 결절인가?' 바이너리로 변환 가능하니 이 마스크를 Ct 객체에 삽입
# 결절 후보 중 실제 결절만 포함되는 리스트로 필터링한 후 애노테이션 마스크 만든 후 결절 마스크 복셀이 적어도 하나 이상 들어 있는 중복된 값이 없는 배열 인덱스로 만듦
# 추후에 검증용 데이터를 만들 때도 이 배열 인덱스 사용
# dsets.py:99
def __init__(self, series_uid):
    # 116행
    candidateInfo_list = getCandidateInfoDict()[self.series_uid]

    self.positiveInfo_list = [
        candidate_tup
        for candidate_tup in candidateInfo_list
        if candidate_tup.isNodule_bool # 결절 필터
    ]
    self.positive_mask = self.buildAnnotationMask(self.positiveInfo_list)
    self.positive_indexes = (self.positive_mask.sum(axis=(1,2)) # 각 단면마다 몇 개의 복셀 마스크가 켜져 있는지를 1차원 벡터에 담아 넘기는 코드
                             .nonzero()[0].tolist()) # 카운트가 0이 아닌 마스크 단면의 인덱스를 얻어 리스트로 만듦

In [ ]:
# getCandidateInfoDict 함수의 정의는 getCandidateInfoList 함수에서와 같이 동일한 정보를 재구성 하는 함수로서 series_uid로 그룹지어져 있음
# dsets.py:87
@functools.lru_cache(1) # Ct 초기화 루틴 병목되지 않게 하는 함수
def getCandidateInfoDict(requireOnDisk_bool=True):
    candidateInfo_list = getCandidateInfoList(requireOnDisk_bool)
    candidateInfo_dict = {}

    for candidateInfo_tup in candidateInfo_list:
        candidateInfo_dict.setdefault(candidateInfo_tup.series_uid,
                                      []).append(candidateInfo_tup) # dict에서 시리즈 UID에 해당하는 후보 리스트 만들거나 없으면 빈 리스트 넘기고 candidateInfo_tup 이어 붙임

    return candidateInfo_dict

In [ ]:
# CT와 함께 마스크도 캐싱하기
# 앞선 단원들에서 CT의 일부만 읽을 때 모든 CT 데이터 읽어 파싱할 필요 없도록 결절 후보 주위로 중심이 잡힌 여러 CT를 캐싱했음
# positive_mask에 대해서도 동일한 작업 필요하므로 Ct.getRawCandidate 함수에서 이를 반환하도록 만듦
# dsets.py:178
def getRawCandidate(self, center_xyz, width_irc):
    center_irc = xyz2irc(center_xyz, self.origin_xyz, self.vxSize_xyz,
                         self.direction_a)

    slice_list = []

    # 203행
    ct_chunk = self.hu_a[tuple(slice_list)]
    pos_chunk = self.positive_mask[tuple(slice_list)] # 새로 추가

    return ct_chunk, pos_chunk, center_irc # 새 값 리턴

In [ ]:
# CT 열어 결절 마스크가 포함된 특정 행의 후보를 얻고 여러 CT데이터와 마스크 센터 정보 등을 반환하기 전에 값을 정리하는 getCtRawCandidate 함수 호출을 통해 디스크에 캐시
# prepcache 스크립트가 우리를 위해 값을 미리 계산하고 저장하여 훈련을 빨리 진행할 수 있도록 도움
# dsets.py:212
@raw_cache.memoize(typed=True)
def getCtRawCandidate(series_uid, center_xyz, width_irc):
    ct = getCt(series_uid)
    ct_chunk, pos_chunk, center_irc = ct.getRawCandidate(center_xyz,
                                                         width_irc)
    ct_chunk.clip(-1000, 1000, ct_chunk)
    return ct_chunk, pos_chunk, center_irc

In [29]:
# 애노테이션 데이터 정제하기
# 애노테이션 데이터를 더 나은 방법으로 선별하는 것
# 현재 candidates.csv에 나열된 몇몇 후보 데이터는 파일에 여러 번 나타남
# 결절 데이터는 중복인데 애노테이션은 다른 경우처럼 완전하게 동일한 데이터가 아님
# 아마 원래 레이블 작업자의 애노테이션이 파일에 입력되기 전에 지워지지 않았기 때문
# 동일한 결절인데 다른 CT 단면에서 애노테이션이 달렸으므로 분류기에는 도움되는 정보일 수 있음

In [ ]:
# 속임수 사용해 클린업된 annotation.csv 파일 제공
# 중복이 없도록 추려놓은 파일로 새 애노테이션 파일로부터 결절 추출하도록 getCandidateInfoList 함수 업데이트 가능
# 새 애노테이션 정보를 돌면서 실제 결절 찾고 CSV 리더 사용해 데이터 타입 변환 후 CandidateInfoTuple 자료 구조에 넣으면 됨
# dsets.py:43
candidateInfo_list = []
with open('data/part2/luna/annotations_with_malignancy.csv', "r") as f:
    for row in list(csv.reader(f))[1:]: # 애노테이션 파일의 한 행은 각기 하나의 결절을 나타냄
        series_uid = row[0]
        annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
        annotationDiameter_mm = float(row[4])
        isMal_bool = {'False': False, 'True': True}[row[5]]

        candidateInfo_list.append( # 리스트에 레코드 추가
            CandidateInfoTuple(
                True, # isNodule_bool
                True, # hasAnnotation_bool
                isMal_bool,
                annotationDiameter_mm,
                series_uid,
                annotationCenter_xyz,
            )
        )

In [ ]:
# candidates.csv의 결절 후보에 대해 루프를 돈 것과 유사
# 결절이 아닌 경우 결절 정보 영역은 단순히 False나 0 넣음
# dsets.py:62
with open('data/part2/luna/candidates.csv', "r") as f:
    for row in list(csv.reader(f))[1:]: # candidates 파일 한 줄마다
        series_uid = row[0]

        if series_uid not in presentOnDisk_set and requireOnDisk_bool:
            continue

        isNodule_bool = bool(int(row[4])) # 결절이 아닌 경우만
        candidateCenter_xyz = tuple([float(x) for x in row[1:4]]) # 후보 레코드 넣기

        if not isNodule_bool:
            candidateInfo_list.append(
                CandidateInfoTuple(
                    False, # isNodule_bool
                    False, # hasAnnotation_bool
                    False, # isMal_bool
                    0.0,
                    series_uid,
                    candidateCenter_xyz,
                )
            )

In [30]:
# 추가한 hasAnnotation_bool과 isMal_bool 플래그 외에 새로운 애노테이션 데이터 공간이 마련되어 전과 동일하게 사용 가능

#### .4 Luna2dSegmentationDataset 구현

In [31]:
# 전까지와 다른 방법으로 훈련 데이터와 검증 데이터 분리
# 검증 데이터를 위해 일반적인 기본 클래스처럼 동작하는 클래스와, 랜덤화와 샘플 잘라내기를 수행해서 훈련셋을 위해 이 기본 클래스를 서브클래싱하는 클래스, 두 개의 클래스 만들 것임
# 다소 복잡한 방식이지만 훈련 샘플을 랜덤화하고 선택하는 등의 작업을 단순화하는 역할하며, 어떤 코드 경로가 훈련과 검증 둘 다에 영향을 미치는지와 어떤 코드 경로가 훈련에만 국한되는지도 명확하게 파악하게 해줌

In [32]:
# 만들 데이터는 2차원의 CT 단면이며 여러 채널을 가짐
# 추가 채널에는 인접한 CT 단명 정보가 저장됨
# 단면들을 합치기 위해 각 단면을 단일 채널로 취급하고 여러 채널 가진 2차원 이미지 만들어 CT 스캔의 각 단면을 RGB 이미지의 각 컬러 채널처럼 간주함
# CT의 각 입력 단면은 스택처럼 쌓여 2차원 이미지처럼 처리됨

In [ ]:
# 검증 과정에서는, 검증용 CT 단면당 하나의 양성 마스크 가진 샘플 만듦
# CT 스캔마다 단면 수가 다를 수 있으므로 각 CT 스캔의 크기와 양성 마스크를 디스크에 캐싱하는 함수 만듦
# 데이터 추출 과정은 모델 훈련 시작전에 돌려야 하는 precache.py 스크립트 실행 중 진행됨
# dsets.py:220
@raw_cache.memoize(typed=True)
def getCtSampleSize(series_uid):
    ct = Ct(series_uid)
    return int(ct.hu_a.shape[0]), ct.positive_indexes

In [33]:
# Luna2dSegmentationDataset.__init__ 메소드의 대부분은 전에 봤던 것과 유사하며 augmentation_dict와 유사한 contextSlices_count 파라미터가 추가됨

In [ ]:
# 훈련인지 검증인지 구별하는 플래그 다루는 부분도 수정
# 개별 결절로 훈련하는 경우는 이제 없으므로 연속된 리스트 전체를 받아 훈련셋과 검증셋으로 나눠야 함
# 결절 후보가 포함된 전체 CT 스캐은 이제 훈련셋에 있거나 검증셋에 있는 것으로 분리
# dsets.py:242
if isValSet_bool:
    assert val_stride > 0, val_stride
    self.series_list = self.series_list[::val_stride] # 연속되어 들어잇는 모든 데이터를 0부터 시작하여 val_stride번째만 남김
    assert self.series_list
elif val_stride > 0:
    del self.series_list[::val_stride] # 훈련인 경우 모든 val_stride번째 요소 제거
    assert self.series_list

In [34]:
# 훈련 결과 검증을 위해 두 가지 모드 지원하도록 제작
# fullCt_bool이 참인 경우 데이터셋의 모든 CT 단면 사용하는 모드: CT에 대한 사전 지식 없이 시작하는 셈이므로 엔드투엔드 파이프라인의 성능 평가할 때 유용
# 양성 마스크 있는 CT 단면에 대해서만 훈련 시에 검증하는 모드

In [ ]:
# 특정 CT 시리즈만 사용하기 원할 경우 조건에 맞는 시리즈 UID를 루프 돌면서 전체 단면 수와 조건에 맞는 단면 수를 세어서 얻음
# dsets.py:250
self.sample_list = []
for series_uid in self.series_list:
    index_count, positive_indexes = getCtSampleSize(series_uid)

    if self.fullCt_bool:
       self.sample_list += [(series_uid, slice_ndx) # range를 사용해 sample_list를 전체 단면으로 확장
                             for slice_ndx in range(index_count)]
    else:
       self.sample_list += [(series_uid, slice_ndx) # 관심 있는 단면만 사용
                             for slice_ndx in positive_indexes]

In [35]:
# 이런 방식은 상대적으로 검증을 빠르게 수행하여 참 양성과 거짓 음성 통계를 얻을 수 있음
# 다만 이는 통계값이 나머지 단면에 대해 거짓 양성과 참 음성 통계가 비슷할 것이라는 가정을 하는 셈임

In [ ]:
# series_uid 값을 한 세트 확보하면 포함된 series_uid를 가지는 결절 후보만 candidateInfo_list에서 골라낼 수 있음
# 양성 후보만 포함하는 리스트를 만들어 훈련 샘플로 사용
# dsets.py:261
self.candidateInfo_list = getCandidateInfoList() # 캐시됨

series_set = set(self.series_list) # 빠른 참조를 위해 셋 구성
self.candidateInfo_list = [cit for cit in self.candidateInfo_list
                            if cit.series_uid in series_set] # 셋에 없는 후보 거르기

self.pos_list = [nt for nt in self.candidateInfo_list
                    if nt.isNodule_bool] # 나중의 데이터 밸런싱을 위해 실제 결절 리스트만 추출

In [ ]:
# __getitem__ 구현은 특정 샘플의 추출을 쉽게 만들어 주는 함수로 로직의 많은 부분을 위임하여 깔끔해짐
# 핵심적으로 샘플 데이터 추출은 세 가지 다른 형태로 이루어짐
# 1. series_uid와 ct_ndx로 지정되는 CT 단면 전체 데이터
# 2. 결절 주위를 잘라낸 훈련 데이터
# 3. DataLoader는 정수 ndx로 샘플 요청하고 데이터셋은 이 값에 따라 훈련용 혹은 검증용 데이터 반환

In [36]:
# 기본 클래스 혹은 서브 클래스의 __getitem__ 함수는 ndx 정수값에 따라 전체 단면이나 훈련용 크롭 샘플 등 적절한 데이터 마련함
# 검증셋의 __getitem__은 실제 작업을 수행하는 함수를 호출하는 역할만 수행하며, 호출전 샘플 리스트의 인덱스를 래핑하는 역할로 에포크 크기와 실제 샘플 수와의 연관성 분리시킴
# dsets.py:281
def __getitem__(self, ndx): # 모듈러 연산이 래핑 수행하는 셈
    series_uid, slice_ndx = self.sample_list[ndx % len(self.sample_list)]
    return self.getitem_fullSlice(series_uid, slice_ndx)

In [ ]:
# getitem_fullSlice 메소드에 추가 기능 구현
# dsets.py
def getitem_fullSlice(self, series_uid, slice_ndx):
        ct = getCt(series_uid)
        ct_t = torch.zeros((self.contextSlices_count * 2 + 1, 512, 512)) # 출력 공간 미리 할당

        start_ndx = slice_ndx - self.contextSlices_count
        end_ndx = slice_ndx + self.contextSlices_count + 1
        for i, context_ndx in enumerate(range(start_ndx, end_ndx)):
            context_ndx = max(context_ndx, 0) # ct_a 경계 넘어서면 처음 or 마지막 단면 중복됨
            context_ndx = min(context_ndx, ct.hu_a.shape[0] - 1)
            ct_t[i] = torch.from_numpy(ct.hu_a[context_ndx].astype(np.float32))

        ct_t.clamp_(-1000, 1000)

        pos_t = torch.from_numpy(ct.positive_mask[slice_ndx]).unsqueeze(0)

        return ct_t, pos_t, ct.series_uid, slice_ndx

In [37]:
# 이렇게 함수로 분리해두는 것은 나중에 특정 시리즈UID나 위치에 대한 단면이 필요할 수 있다는 의미
# __getitem__에 정수로 된 인덱스들만 전달하면 리스트에서 샘플 얻을 수 있음
# 반환하는 튜플에서 ct_t와 pos_t 외의 나머지는 디버깅과 출력을 위한 것으로 훈련에는 필요하지 않음

#### .5 훈련 데이터와 검증 데이터 설계

In [39]:
# 왜 훈련 데이터가 검증 데이터와 달라 보이는지 설명
# CT 단면 전체 대신, 양성 후보 주위 64*64 패치(결절이 가운데 위치한 96*96 패치에서 랜덤하게 잘라낸 것) 훈련
# 이 데이터 앞뒤에 위치하는 세 개 단면까지  2차원 세그멘테이션에 추가 '채널'로 포함함

In [40]:
# 훈련을 안정적으로 동작시키고 빠르게 수렴하게 만들기 위함
# 몇 가지 실험으로 64*64의 일부 랜덤 크롭 사용하는 접근 방식이 잘 동작하는 사실 발견, 이 책에서 적용하기로 함
# 타 프로젝트 진행하면 이런 실험이 필요할 것

In [41]:
# 단면 전체를 사용한 훈련이 불안정한 이유는 클래스 밸런싱 이슈가 원인으로 보임
# CT 단면 크기에 비해 결절이 너무 작아 양성 샘플이 음성 샘플로 뒤덮히는 효과를 지니는 것으로 보임
# 크롭된 데이터로 훈련할 때 양성 픽셀 수는 동일하게 유지하며 음성 픽셀 수는 줄여주는 방식임

In [42]:
# 우리의 세그멘테이션 모델은 픽셀투픽셀 모델이며, 받는 이미지 크기도 다양하기 때문에 다른 차원의 훈련과 검증에서도 잘 동작할 수 있음
# 검증에서는 가중치가 같은 동일한 컨볼루션 사용하고 크기가 더 큰 픽셀 집합에 적용하는 차이점만 존재

In [43]:
# 이 방식의 주의점 중 하나는 검증셋의 음성 픽셀이 양성보다 훨씬 많기 때문에 모델 검증 시 아주 높은 거짓 양성 비율을 보임
# 따라서 세그멘테이션 모델은 속기 쉬움

#### .6 TrainingLuna2dSegmentationDataset 구현

In [ ]:
# 연속된 데이터와 단면이 아닌 정확한 중심 위치가 필요하기에 pos_list로부터 만든 샘플로 후보 정보 튜플을 인자로 하여 getItem_trainingCrop을 호출하는 것 외에는 검증셋 __getitem__과 같은 훈련셋용__getitem__
# dsets.py:320
def __getitem__(self, ndx):
    candidateInfo_tup = self.pos_list[ndx % len(self.pos_list)]
    return self.getitem_trainingCrop(candidateInfo_tup)

In [44]:
# getItem_trainingCrop 구현하려면 분류 훈련에서 사용한 것과 유사한 getCtRawCandidate 함수 사용해야 함
# 크기 다른 크롭된 데이터 전달하지만 함수는 ct.positive_mask의 크롭 배열을 추가적으로 반환하는 부분 외에는 바뀌지 않음

In [ ]:
# pos_a는 실제 세그멘테이션 진행하는 중앙부 단면으로 제한하고 getRawCandidate로부터 얻은 96*96에서 랜덤으로 잘라낸 64*64 크롭 만듦
# 모든 작업 수행 후 검증 데이터셋과 동일한 항목으로 튜플 만들어 반환
# dsets.py:324
def getitem_trainingCrop(self, candidateInfo_tup):
    ct_a, pos_a, center_irc = getCtRawCandidate( # 경계 데이터가 추가로 더 들어간 후보 얻음
        candidateInfo_tup.series_uid,
        candidateInfo_tup.center_xyz,
        (7, 96, 96),
    )
    pos_a = pos_a[3:4] # 출력 채널인 세 번째 차원의 단면 유지

    row_offset = random.randrange(0,32) # 0과 31 사이의 임의의 숫자 두 개로 CT와 마스크 크롭
    col_offset = random.randrange(0,32)
    ct_t = torch.from_numpy(ct_a[:, row_offset:row_offset+64,
                                 col_offset:col_offset+64]).to(torch.float32)
    pos_t = torch.from_numpy(pos_a[:, row_offset:row_offset+64,
                                 col_offset:col_offset+64]).to(torch.long)

    slice_ndx = center_irc.index

    return ct_t, pos_t, candidateInfo_tup.series_uid, slice_ndx

#### .7 GPU에서 돌리는 증강

In [ ]:
# 딥러닝 모델 훈련 시 가장 고려할 점은 훈련 파이프라인에서 병목 지점을 피하는 것
# 업그레이드 비용 크거나 어려운 지점에서의 리소르 병목 현상이 발생하는지, 리소스가 낭비되고 있지 않은지 확인하는 것이 요령

In [ ]:
# 병목 지점 확인하는 일반적인 방법
# 1. 데이터 로딩 파이프라인에서 저수준 I/O나 일단 램에 올라온 이후 데이터 압축 해제 지점
# 2. 로딩한 데이터를 CPU가 전처리하는 지점. 주로 데이터 정규화나 증강에 해당
# 3. GPU에서 동작하는 훈련 루프. 물리적 저장 공간이나 CPU보다 GPU 비용이 딥러닝 시스템에서 가장 높기 때문에 이 부분의 병목은 정상
# 4. 일반적이진 않지만 CPU와 GPU 간 메모리 대역폭에서 병목 발생. 전송되는 데이터 크기에 비해 GPU 작업이 많지 않다는 뜻

In [45]:
# 따라서 데이터 증강 GPU에서 수행할 예정
# 이를 위해 nn.Module의 서브클래스와 비슷한 두 번째 모델 만들 것
# 주요 차이점으로는 모델을 통한 역전파 기울기는 중요치 않으며, forward 메소드는 완전히 다른 일 하게 됨
# 실제 증강 루틴에는 약간의 수정 필요
# 앞서 만든 다른 모델처럼 텐서를 받아 다른 텐서 만듦

In [ ]:
# 모델의 __init__은 이전 장과 동일한 형태의 데이터 증강을 위한 flip과 offset 인자를 받아 self에 할당
# model.py:56
class SegmentationAugmentation(nn.Module):
    def __init__(
            self, flip=None, offset=None, scale=None, rotate=None, noise=None
    ):
        super().__init__()

        self.flip = flip
        self.offset = offset
        self.scale = scale
        self.rotate = rotate
        self.noise = noise

In [ ]:
# 증강 forward 메소드는 입력과 레이블을 받아 affine_grid와 grid_sample 호출에 사용할 transform_t 텐서 만듦
# model.py:68
def forward(self, input_g, label_g):
    transform_t = self._build2dTransformMatrix()
    transform_t = transform_t.expand(input_g.shape[0], -1, -1) # 2차원 데이터 증강
    transform_t = transform_t.to(input_g.device, torch.float32)
    affine_t = F.affine_grid(transform_t[:,:2], # 변환 대상의 첫 차원은 배치 영역이고 각 배치 아이템의 3*3 행렬 중 첫 행과 둘째 행만 사용
            input_g.size(), align_corners=False)

    augmented_input_g = F.grid_sample(input_g,
            affine_t, padding_mode='border',
            align_corners=False)
    augmented_label_g = F.grid_sample(label_g.to(torch.float32),
            affine_t, padding_mode='border',
            align_corners=False) # CT와 마스크에 동일한 변환이 적용되므로 같은 그리드 사용. grid_sample은 부동소수점만 사용하므로 여기서 변환

    if self.noise:
        noise_t = torch.randn_like(augmented_input_g)
        noise_t *= self.noise

        augmented_input_g += noise_t

    return augmented_input_g, augmented_label_g > 0.5 # 반환 전 마스크를 0.5와 비교해 불리언으로 변환. grid_sample이 결과적으로 분수 값이 되는 보간법

In [ ]:
# 사용하는 변환 행렬을 실제로 만들 _build2dTransformMatrics 함수
# model.py:90
def _build2dTransformMatrix(self):
    transform_t = torch.eye(3) # 3*3 행렬 만들고 나중에 마지막 행 버림

    for i in range(2): # 2차원 데이터 증강 중
        if self.flip:
            if random.random() > 0.5:
                transform_t[i,i] *= -1

        if self.offset:
            offset_float = self.offset
            random_float = (random.random() * 2 - 1)
            transform_t[2,i] = offset_float * random_float

        if self.scale:
            scale_float = self.scale
            random_float = (random.random() * 2 - 1)
            transform_t[i,i] *= 1.0 + scale_float * random_float

    if self.rotate:
        angle_rad = random.random() * math.pi * 2 # 라디언으로 랜덤 각 만들기. 범위는 0..2{pi}
        s = math.sin(angle_rad)
        c = math.cos(angle_rad)

        rotation_t = torch.tensor([ # 처음 두 차원에 대한 랜덤 각으로 2차원 회전을 위한 회전 행렬 정의
            [c, -s, 0],
            [s, c, 0],
            [0, 0, 1]])

        transform_t @= rotation_t # 파이썬 행렬곱 연산자로 변환 행렬 회전

    return transform_t

In [46]:
# 2차원 데이터 다룰 때와 약간 다른 점 빼면 GPU 증강 코드는 CPU 증강 코드와 매우 흡사
# 주로 핵심 구현이 아닌 곳에서 차이 발생하며 이를 활용하여 nn.Module 서브클래스로 래핑하였음

## 6절 - 세그멘테이션을 위한 훈련 스크립트 업데이트

In [1]:
# 새 데이터로 새로운 모델 훈련할 것임

In [2]:
# 12장의 훈련 코드에서 출력에 영향 미치는 세 가지 업데이트
# 새 모델 인스턴트, 다이스 손실이라는 새로운 손실값, Adam 옵티마이저

In [3]:
# 작업 설정을 위해 준비할 것
# 텐서보드에서 세그멘테이션을 시각적으로 조사하기 위해 이미지 로깅, 텐서보드에서 더 많은 메트릭 로깅 수행, 검증 결과에 따라 제일 좋은 모델 저장

#### .1 세그멘테이션과 증강 모델 초기화

In [ ]:
# UNetWrapper 클래스 사용하여 설정 파라미터 initModel메소드에 전달
# training.py:133
def initModel(self):
        segmentation_model = UNetWrapper(
        in_channels=7,
        n_classes=1,
        depth=3,
        wf=4,
        padding=True,
        batch_norm=True,
        up_mode='upconv',
    )

    augmentation_model = SegmentationAugmentation(**self.augmentation_dict)

    # 154행
    return segmentation_model, augmentation_model

In [ ]:
# UNet 입력용으로 일곱 개의 입력 채널 존재
# 3+3 개의 콘텍스트 단면과 세그멘테이션하기 위해 집중하고 있는 하나의 단면에 해당하는 채널
# depth 파라미터는 U 모양의 깊이 제어
# 각 다운샘플링 연산으로 깊이는 1씩 증가
# wf=4는 첫 계층이 2**wf==16개 필터를 가진다는 것을 의미하며 다운 샘플링 한 번에 두 배씩 증가함을 알 수 있음
# 입력과 같은 축력 이미지 얻기 위해 컨볼루션 덧대져야 하고 신경망 내 각 활성 함수 뒤에는 배치 정규화 필요
# 업샘플링 함수는 nn.ConvTransPose2d에 구현된 것처럼 업컨볼루션 계층이여야 함

#### .2 아담 옵티마이저 사용하기

In [4]:
# 아담 옵티마이저는 모델 훈련 시 SGD 대체 가능
# 각 파라미터별로 학습률 관리할 수 있고 훈련 진행하며 학습률 자동으로 조정
# 자동 조정으로 적당한 학습률 찾아내므로 사용 시 별도의 학습률을 지정할 필요 없음
# Adam 초기화 코드
# training.py:156
def initOptimizer(self):
    return Adam(self.segmentation_model.parameters())

In [5]:
# 대부분 프로젝트에서 아담은 꽤 훌륭한 옵티마이저
# 변형으로 아담맥스, R아담, 레인저 등 각각의 장단점 존재

#### .3 다이스 손실

In [6]:
# 다이스 손실(Dice loss)로 알려진 쇠렌센 다이스 계수(Sorensen-Dice coefficient)는 세그멘테이션 작업에서 일반적으로 사용하는 손실 메트릭
# 픽셀당 크로스엔트로피 손실 계산할 때 다이스 손실 사용하면 전체 이미지에서 작은 일부분이 양성인 경우도 처리하므로 장점 존재
# CT 스캔 대부분의 영역이 결절이 아니므로 이 경우에 해당

In [7]:
# 쇠렌센 다이스 계수는 정확하게 세그먼트된 픽셀에 대해 예측 영역과 실제 픽셀을 합한 영역과의 비율에 대한 값
# 픽셀 단위의 F1 점수 사용하는 셈

In [8]:
# label_g는 사실상 불리언 마스크이기 때문에 예측 결과에 곱하여 참 양성 얻을 수 있음
# 여기서 prediction_devtensor는 불리언으로 취합하지 않음
# 이와 함께 정의된 손실값은 미분 불가하지만 참 양성의 수를 실측값이 1인 픽셀에 대해 예측한 값의 합으로 바꿀 수 있음
# 예측값이 1에 가까울수록 다이스 점수는 불연속값이나 사용할 때와 동일한 값으로 수렴하게 됨
# 하지만 종종 예측값은 0.4~0.6 범위의 불확실한 값이 될것임
# 이런 애매한 값은 0.5보다 큰쪽 혹은 작은쪽으로 떨어지는지에 상관없이 기울기 조정에 동일한 양만큼 기여함
# 이처럼 다이스 계수 중 연속적인 예측값 활용하는 경우를 소프트 다이스라고 함

In [ ]:
# 손실을 최소화하고 싶어 비율값에서 1을 빼면 손실 함수의 기울기가 뒤집어지므로 많이 중첩되는 경우에는 손실 작아지고 적게 중첩되는 경우엔 손실 커짐
# training.py:315
def diceLoss(self, prediction_g, label_g, epsilon=1):
    diceLabel_g = label_g.sum(dim=[1,2,3]) # 배치 차원 제외한 나머지 모두 합해서 양성 레이블, 경도의 양성 감지, 경도의 정확한 양성을 각 배치 아이템별로 구함
    dicePrediction_g = prediction_g.sum(dim=[1,2,3])
    diceCorrect_g = (prediction_g * label_g).sum(dim=[1,2,3])

    diceRatio_g = (2 * diceCorrect_g + epsilon) \
        / (dicePrediction_g + diceLabel_g + epsilon) # 다이스 비율. 예측이나 레이블 둘 다 없는 경우를 해결하기 위해 분자와 분모 모두에 1 더함

    return 1 - diceRatio_g # 손실값 만들기 위해 '1 - 다이스 비율' 반환. 값은 작을수록 좋음

In [ ]:
# computeBatchLoss 함수가 self.diceLoss를 호출하도록 업데이트
# 훈련 샘플에 대해 일반 다이스 손실값 한 번 계산하고 label_g에 포함된 픽셀에 대해서 한 번 계산
# 예측값에 레이블을 곱해서 음성인 픽셀이 '정확하게 맞은' 의사(pseudo) 예측을 얻을 수 있음
# 손실값을 만드는 유일한 픽셀은 거짓 음성 픽셀임
# 현 프로젝트에서는 재현율이 매우 중요하기 때문에 이런 방법이 유효함 -> 처음부터 종양을 제대로 인식하기 못하면 구별 자체를 못하기 때문
# training.py:282
def computeBatchLoss(self, batch_ndx, batch_tup, batch_size, metrics_g,
                     classificationThreshold=0.5):
    input_t, label_t, series_list, _slice_ndx_list = batch_tup

    input_g = input_t.to(self.device, non_blocking=True) # GPU 이동
    label_g = label_t.to(self.device, non_blocking=True)

    if self.segmentation_model.training and self.augmentation_dict: # 훈련 때는 인자가 필요하고 검증때는 넘어감
        input_g, label_g = self.augmentation_model(input_g, label_g)

    prediction_g = self.segmentation_model(input_g) # 세그멘테이션 모델 실행

    diceLoss_g = self.diceLoss(prediction_g, label_g) # 다이스 손실값 적용
    fnLoss_g = self.diceLoss(prediction_g * label_g, label_g)

    # 313행
    return diceLoss_g.mean() + fnLoss_g.mean() * 8

In [9]:
# 손실값 가중치
# 12장처럼 동일한 균형을 맞추는 방식을 사용하여 훈련 샘플 크롭을 통해 양성 픽셀이 아닌 경우를 줄임
# 하지만 높은 재현율이 중요한 상황이므로 이런 점을 반영하는 손실값을 훈련에 사용할 수 있게 함

In [10]:
# 나머지 부류에 비해 한 부류가 더 선호되는 가중치 손실(weighted loss) 만들 것임
# fnLoss_g에 8 곱하면 양성 픽셀 전체 개체는 전체 개체보다 8배 중요하게 설정됨(diceLoss_g까지 센다면 9)
# 양성 마스크에 해당하는 부분이 전체 크롭(64*64)에 비해 훨씬 작으므로 개별 양성 픽셀은 역전파시 반대로 많은 영향력 가지게 된다는 의미

In [11]:
# 일반 다이스 손실에서 정확하게 예측된 음성 픽셀은 거짓 음성 손실에서 한 개의 정확한 픽셀를 얻기 위해 버림
# 일반 다이스 손실은 거짓 음성 손실에 대한 엄격한 초집합이며 들어가는 픽셀은 참 음성이여야 함(모든 참 양성 픽셀은 거짓 음성 손실값에 이미 포함되어 있으므로 이 집합에 들어갈 픽셀은 없음)

In [12]:
# 좋은 재현율을 얻기 위해 상당량의 참 음성 픽셀을 희생할 것이므로 많은 거짓 양성이 존재한다고 가정해야 함
# 재현율이 매우 중요한 프로젝트이므로 하나의 거짓 음성이 존재하는 것보다 차라리 여러 개의 거짓 양성이 존재하는 편이 나음

In [13]:
# 이런 접근 방법은 아담을 사용할때만 유효함
# SGD 사용하면 과하게 예측하려는 성향은 모든 픽셀을 양성으로 만들 수 있음
# 아담의 학습률 미세 튜닝 능력은 이 경우에서 거짓 음성 손실이 우세하지 않도록 제어하는 역할함

In [ ]:
# 메트릭 수집
# 재현율을 높이기 위해 일부 숫자를 왜곡한 상태 확인
# computeBatchLoss 분류에서 각 샘플별로 메트릭에 사용할 다양한 값 계산함
# 전체 세그멘테이션 결과와 유사한 값도 계산함
# 참 양성을 비록한 기타 메트릭은 이전에 logMetrics에서 계산되었음
# 하지만 결과 데이터의 크기(검증셋의 개별 CT 단면은 25만 개의 픽셀 가짐)로 인해 요약 통계는 computeBatchLoss 함수에서 실시간으로 계산해야 함
# training.py:297
start_ndx = batch_ndx * batch_size
end_ndx = start_ndx + input_t.size(0)

with torch.no_grad():
    predictionBool_g = (prediction_g[:, 0:1]
                        > classificationThreshold).to(torch.float32) # 확실한 다이스 값을 위해 예측값을 경계값으로 분리하고 이후 곱셈 연산을 위해 부동소수점으로 변환

    tp = (     predictionBool_g *  label_g).sum(dim=[1,2,3]) # 참 양성, 거짓 양성, 거짓 음성 계산은 다이스 손실 계산과 유사
    fn = ((1 - predictionBool_g) *  label_g).sum(dim=[1,2,3])
    fp = (     predictionBool_g * (~label_g)).sum(dim=[1,2,3])

    metrics_g[METRICS_LOSS_NDX, start_ndx:end_ndx] = diceLoss_g # 나중에 사용할 목적으로 메트릭을 큰 텐서에 보관. 배치의 개별 아이템에 대한 값임
    metrics_g[METRICS_TP_NDX, start_ndx:end_ndx] = tp
    metrics_g[METRICS_FN_NDX, start_ndx:end_ndx] = fn
    metrics_g[METRICS_FP_NDX, start_ndx:end_ndx] = fp

In [ ]:
# 예측값과 레이블을 곱하여 참 양성 계산 가능
# 정확한 예측값이 중요치 않으므로(경계값 넘기만 하면 픽셀 값은 문제되지 않으며 이를 결절 후보로 생각할 것임) 경계값 0.5와 비교한 predictionBool_g 만들 것임

#### .4 이미지를 텐서보드에 넣기

In [14]:
# 세그멘테이션 작업에서 좋은 점 중 하나는 출력 결과를 쉽게 육안으로 확인 가능하다는 점임
# 모델이 잘 진행하는지 훈련이 더 필요한지 궤도를 벗어났는지 추가 훈련이 불필요한지 판단 가능
# 결과를 이미지로 만드는 방법은 많고 표시하는 방법도 다양함
# 텐서보드는 이런 류의 데이터를 잘 지원하며 우리 훈련에 통합된 텐서보드의 SummaryWriter 인스턴스 있으므로 텐서보드로 확인

In [15]:
# 메인 애플리케이션 클래스에 logImages 함수를 추가하고 훈련 데이터와 검증 데이터 로더 양쪽에서 호출
# 훈련 루프도 바꿔서 처음 샘플과 이후 모든 5번째 에포크에 해당하는 샘플에 대해 검증과 이미지 로깅 실행
# 새로 만든 상수 validation_cadence와 에포크 수 비교하여 만들 수 있음
# 훈련 때 확인할 사항: 오래 기다릴 필요 없이 모델 훈련이 어떻게 진행되는지 대략적으로 알아내기 / GPU 사이클 대부분을 검증이 아닌 훈련에 사용하기 / 검증셋에서 좋은 성능을 보이고 있는지 확인하기

In [ ]:
# 첫 번째 포인트는 에포크를 상대적으로 짧게 잡아 logMetrics를 더 자주 호출하는 것
# 두 번째 포인트는 doValidation 호출 전에 훈련을 좀 더 오래 수행하는 것
# 세 번째 포인트는 doValidation을 규칙적으로 호출하는 것
# 검증을 최초에 한 번 수행하고 이후 매 다섯 번째 에포크에서 수행하는 것으로 모든 목적 달성 가능
# 훈련 프로세스의 초기 신호를 얻고 나머지 시간의 상당 부분을 훈련에 사용한 후 작업 진행 과정에서 검증셋에 대한 주기적인 체크인 수행
# training.py:210
def main(self):
    # 217행
    self.validation_cadence = 5
        for epoch_ndx in range(1, self.cli_args.epochs + 1): # 에포크 도는 제일 바깥 루프
            # 228행
            trnMetrics_t = self.doTraining(epoch_ndx, train_dl) # 각 에포크당 훈련
            self.logMetrics(epoch_ndx, 'trn', trnMetrics_t) # 각 에포크 훈련에서 스칼라 메트릭을 로깅

        if epoch_ndx == 1 or epoch_ndx % self.validation_cadence == 0: # 검증은 매 cadence번째에서 수행
            # 239행
            self.logImages(epoch_ndx, 'trn', train_dl) # 모델 검증하고 이미지 로깅
            self.logImages(epoch_ndx, 'val', val_dl)

In [16]:
# 이미지 로깅 구조화는 정답이 없음
# 몇 개의 CT를 훈련셋과 검증셋 양쪽에서 가져와야 함
# 각 CT에 대해 6개의 동일한 간격을 가지는 단면을 골라서 처음부터 끝까지 나열하고 실측 값과 모델 출력을 나란히 보여줄 것
# prediction 이미지에 작게 슬라이더가 표시된 부분 확인
# 슬라이더로 동일한 레이블에 대한 이전 버전 이미지 확인 가능
# 시간이 지남에 따라 세그멘테이션 출력의 변화 확인할 수 있으면, 디버깅이나 다른 결과 얻도록 조절하기 유용함

In [ ]:
# 이 출력 만드는 코드는 적절한 데이터 로더에서 12개의 연속된 데이터를 가져와 각 시리즈에서 6개의 이미지 얻음
# training.py:326
def logImages(self, epoch_ndx, mode_str, dl):
    self.segmentation_model.eval() # 평가 모드로 설정

    images = sorted(dl.dataset.series_list)[:12] # 데이터 로더를 우회애서 직접 데이터셋을 사용해 얻은 12개의 CT 순서가 섞여있을 수 있으므로 정렬
    for series_ndx, series_uid in enumerate(images):
        ct = getCt(series_uid)

        for slice_ndx in range(6):
            ct_ndx = slice_ndx * (ct.hu_a.shape[0] - 1) // 5 # CT 내에서 동일한 거리 가지는 6개의 단면 선택
            sample_tup = dl.dataset.getitem_fullSlice(series_uid, ct_ndx)

            ct_t, label_t, series_uid, ct_ndx = sample_tup

In [ ]:
# 이후 ct_t를 모델에 넣음
# prediction_a를 화면에 표시하려면 RGB값 가지는 image_a 필요
# 꼼수 사용해 여러 이미지와 마스크를 넣어 범위가 0~2인 데이터 얻어 전체 배열에 0.5 곱해 전체 배열의 값이 원하는 범위 안에 들어오게 만듦
# training.py:346
ct_t[:-1,:,:] /= 2000
ct_t[:-1,:,:] += 0.5

ctSlice_a = ct_t[dl.dataset.contextSlices_count].numpy()

image_a = np.zeros((512, 512, 3), dtype=np.float32)
image_a[:,:,:] = ctSlice_a.reshape((512,512,1)) # 기본으로 흑백 이미지 제공할 수 있도록 CT밀도를 모든 RGB채널에 할당
image_a[:,:,0] += prediction_a & (1 - label_a) # 거짓 양성은 붉은색 표시로 이미지 위에 포개기
image_a[:,:,0] += (1 - prediction_a) & label_a # 거짓 음성은 주황색으로 표시
image_a[:,:,1] += ((1 - prediction_a) & label_a) * 0.5

image_a[:,:,1] += prediction_a & label_a # 참 양성은 녹색으로 표시
image_a *= 0.5
image_a.clip(0, 1, image_a)

In [17]:
# 밀도가 절반 쯤 되는 회색 CT를 만들고, 결절 후보로 예측된 픽셀을 다양한 색으로 포개는 것이 목적
# 거짓 양성과 거짓 음성은 붉은색으로 표시
# 1-label_a는 레이블 뒤집고 prediction_a에 곱하면 예측된 픽셀이 결절 후보 아닌 경우만 얻음
# 거짓 음성은 녹색에 추가될 때 마스크의 강도를 절반으로 하여 주황색으로 만듦
# 결절에 해당하는 곳을 정확하게 예측한 픽셀은 녹색으로 설정

In [ ]:
# 데이터를 다시 0...1이 되도록 정규화하고 고정함(증강 데이터도 표시하는 경우 예상 CT 범위 넘어서는 노이즈에는 반점 보일 수 있음)
# 나머지 데이터는 텐서보드엣 사용하도록 저장
# training.py:361
writer = getattr(self, mode_str + '_writer')
writer.add_image(
    f'{mode_str}/{series_ndx}_prediction_{slice_ndx}',
    image_a,
    self.totalTrainingSamples_count,
    dataformats='HWC',
)

In [18]:
# 위 코드는 writer.add_scalar와 매우 흡사
# data-foramts='HWC' 인자는 텐서보드에게 이미지 내부 축 순서에서 RGB는 세 번째 축임을 알려줌
# 신경망 계층에서 B*C*H*W로 지정하는 경우 많으며, 원한다면 데이터를 텐서보드에 'CHW'로 정의해 바로 넣을 수 있는 점 기억

In [19]:
# 텐서보드 CT 단면 상단 이미지처럼 보여야 하므로 훈련에 사용했던 실측 데이터도 저장(책에서 코드 생략)

#### .5 메트릭 로깅 업데이트

In [ ]:
# 잘 작동하는지 확인하기 위해 각 에포크마다 메트릭 계산하며, 참 양성과 거짓음성, 거짓 양성에 주의
# training.py:400
sum_a = metrics_a.sum(axis=1)
allLabel_count = sum_a[METRICS_TP_NDX] + sum_a[METRICS_FN_NDX]
metrics_dict['percent_all/tp'] = \
    sum_a[METRICS_TP_NDX] / (allLabel_count or 1) * 100
metrics_dict['percent_all/fn'] = \
    sum_a[METRICS_FN_NDX] / (allLabel_count or 1) * 100
metrics_dict['percent_all/fp'] = \
    sum_a[METRICS_FP_NDX] / (allLabel_count or 1) * 100 # 결절 후보로 레이블된 픽셀 수와 비교. 이미지마다 차이가 있어 100%보다 클 수 있음

In [ ]:
# 모델에 점수 매겨 어떤 훈련이 가장 좋은지 확인
# 찾지 못하면 결절인지 구분하는 것이 불가능 하므로 재현율이 최대한 높아야 함
# 특정 에포크에서 F1 점수가 나쁘지 않으면 그 안에서 최대한 재현율을 높임
# training.py:393
def logMetrics(self, epoch_ndx, mode_str, metrics_t):
    score = metrics_dict['pr/recall']

    return score

In [20]:
# 분류용 훈련 루프에 비슷한 코드 추가할 때는 F1 점수 사용

In [ ]:
# 메인 훈련 루프로 돌아가면 훈련 실행 시 지금까지 확인한 best_score 값을 기록하고 최고 점수 알려주는 플래그 포함함
# doValidation 함수를 호출 후 5번째 에포크마다 호출하면서 점수가 최고를 갱신하는지 체크함
# 점수 체크는 이미지 저장 전 수행함
# training.py:210
def main(self):
    best_score = 0.0
    for epoch_ndx in range(1, self.cli_args.epochs + 1): # 에포크 루프
        # 검증을 원할 경우
        # 233행
        valMetrics_t = self.doValidation(epoch_ndx, val_dl)
        score = self.logMetrics(epoch_ndx, 'val', valMetrics_t) # 재현율 취하여 점수 계산
        best_score = max(score, best_score)

        self.saveModel('seg', epoch_ndx, score == best_score) # saveModel 저장 부분만 작성하면 됨. 세 번째 파라미터는 최고인 경우만 기록한다는 의미

#### .6 모델 저장

In [21]:
# 파이토치 내부적으로 torch.save는 파이썬 라이브러리인 pickle 사용하므로 모델 인스턴스 직접 전달해 적절히 저장하도록 동작하지만 유연성을 놓치는 면이 있어 모델 저장에 이상적인 방법은 아님
# 모델의 파라미터만 저장 가능함
# 동일한 모양의 파라미터를 기대하는 어떤 모델도 적용가능해짐
# 파라미터만 저장하는 방식은 모델 전체 저장하는 방식보다 모델 재사용하거나 다르게 섞을 수 있게 해줌

In [ ]:
# 모델 파라미터는 model.state_dict() 함수 사용해 얻을 수 있음
# training.py:480
def saveModel(self, type_str, epoch_ndx, isBest=False):
    # 496행
    model = self.segmentation_model
    if isinstance(model, torch.nn.DataParallel):
        model = model.module # DataParallel 래퍼 있으면 제거함

    state = {
        'sys_argv': sys.argv,
        'time': str(datetime.datetime.now()),
        'model_state': model.state_dict(), # 중요 부분
        'model_name': type(model).__name__,
        'optimizer_state' : self.optimizer.state_dict(), # 모멘텀 등 보관
        'optimizer_name': type(self.optimizer).__name__,
        'epoch': epoch_ndx,
        'totalTrainingSamples_count': self.totalTrainingSamples_count,
    }
    torch.save(state, file_path)

In [ ]:
# 현재 모델이 지금까지 살펴본 점수 중 가장 높다면 state를 .best.state 확장자를 사용해 여벌로 복사
# 나중에 더 높은 점수 가진 모델 나오면 덮어쓰임
# 최상의 파일에만 집중하면 훈련된 모델의 사용자는 훈련 중 만들어지는 각 에포크의 세부 실행 결과 신경쓰지 않아도 됨
# training.py:514
if isBest:
    best_path = os.path.join(
        'data-unversioned', 'part2', 'models',
         self.cli_args.tb_prefix,
         f'{type_str}_{self.time_str}_{self.cli_args.comment}.best.state')
    shutil.copyfile(file_path, best_path)

    log.info("Saved model params to {}".format(best_path))

with open(file_path, 'rb') as f:
    log.info("SHA1: " + hashlib.sha1(f.read()).hexdigest())

In [22]:
# 저장한 모델의 SHA1 값도 출력함
# 상태 디렉토리에 넣은 sys.argv와 타임스탬프처럼 이 값으로 정확하게 작업 중인 모델을 헷갈리지 않고 디버깅 가능

## 7절 - 결과

In [23]:
# 결과 확인
# 훈련 메트릭 출력 결과는 F1 점수 상승, 참 양성 상승, 거짓 음성 및 거짓 양성 감소
# 검증 메트릭 출력 결과는 참 양성 비율은 재현율과 동일, 거짓 양성은 4495%
# 훈련용 크롭 데이터는 2^12이지만 검증 데이터는 2^18픽셀이므로 64배 더 큼 -> 자연스러운 결과

In [24]:
# 재현율이 5에포크와 10에포크에서 안정적이다가 이후 떨어짐 -> 이 시점에서 과적합되는 것 의미
# 300만 개 샘플 이후 훈련 재현율은 상승하지만 검증 재현율 감소하는 경향 보임

In [25]:
# 유넷 아키텍처는 필터와 깊이를 줄여도 용량이 매우 커 빠르게 훈련셋을 암기할 수 있음
# 세그멘테이션 단계에서는 재현율이 최고 우선 순위임
# 이렇게 편향된 상황은 전체 모델 평가하는 것보다 어렵다는 사실 의미함
# 모델 점수는 재현율을 기준으로 매긴 후 사람이 직접 훈련 결과가 의학적으로 쓸만 한지 판단하게 함
# 직접 재현율을 사용하지 않고 다이스 손실로 훈련하고 있으므로 효과 있을 것임

## 8절 - 결론

In [ ]:
# 픽셀투픽셀 세그멘테이션을 위한 모델 구조화하는 새로운 방법 다룸
# 이런 류의 작업에 효과적인 유넷 확인함
# 목적에 맞게 사용할 수 있도록 구현 변경함
# 새 모델의 훈련에 필요한 데이터를 제공하기 위해 기존 데이터셋 수정: 훈련을 위해 작게 크롭된 데이터와 검증을 위한 제한된 단면 집합
# 훈련 루프는 텐서보드에 이미지를 저장하며 데이터셋의 증강 부분을 별도의 모델로 옮겨서 GPU 상에서 동작하도록 만듦
# 훈련 결과 관찰하고 거짓 양성 비율이 기대하던 것과 다르지만 결과적으로 큰 프로젝트에서 주어지는 요구 사항 만족하게 됨

## 9절 - 연습 문제

In [26]:
# 1. (세그멘테이션 훈련에서 사용했던 것처럼)모델 레퍼 접근 방법을 분류 모델을 위한 데이터 증강에 대해 적용해 구현하라.
#   a. 어떤 지점에서 타협이 필요한가?
#   b. 이러한 변경으로 훈련 속도가 어떻게 바뀌었는가?

In [27]:
# 2. 세그멘테이션 Dataset 구현을 변경하여 훈련셋, 검증셋, 테스트셋의 세 가지로 분리하라.
#   a. 테스트셋에 사용한 데이터 비율은 얼마나 되는가?
#   b. 테스트셋과 검증셋에서의 성능은 서로 일치하는 듯이 보이는가?
#   c. 작아진 훈련셋으로 훈련 결과가 얼마만큼 나빠지는가?

In [28]:
# 3. 모델이 결절 여부 외에 악성과 양성도 세그멘테이션하도록 만들어라.
#   a. 메트릭 리포트를 어떻게 바꿔야 할까? 이미지 생성 부분은?
#   b. 어떤 결과를 볼 수 있는가? 세그멘테이션마능로 분류 단계를 건너뛰어도 될 만큼 충분히 좋은가?

In [30]:
# 4. 64*64 크롭과 전체 CT 단면을 섞은 채로 모델을 훈련할 수 있나?

In [31]:
# 5. LUNA(혹은 LIDC) 외에 추가적인 데이터 소스를 찾을 수 있는가?